# LEGACY

## Hyperparameters optimization with hyperopt

In [ ]:
%reload_kedro

In [ ]:
from xgboost import XGBClassifier
from time import time
from sklearn.metrics import accuracy_score
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe, space_eval
from pprint import pprint
import numpy as np
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
import warnings
warnings.filterwarnings("ignore")

## Base

In [ ]:
TARGET_COL = ["label"]
INDEX_COL = "window_nbr"

time_counter_secs = []
scores = []

N_SPLITS = 2

In [ ]:
mt_train_multic = catalog.load("master_table_train_multic")
mt_test_multic = catalog.load("master_table_test_multic")

mt_train_multic = mt_train_multic.set_index(INDEX_COL).sort_index().reset_index(drop=True)
X_train, y_train = mt_train_multic.drop(columns=TARGET_COL), mt_train_multic[TARGET_COL]

X_test_oos = mt_test_multic.set_index(INDEX_COL).drop(columns=TARGET_COL)

In [ ]:
tss = TimeSeriesSplit(n_splits=N_SPLITS)

for i, (train_idx, test_idx) in enumerate(tss.split(X_train), 1):
    if i == N_SPLITS:
        X_train, X_test = X_train.iloc[train_idx], X_train.iloc[test_idx]
        y_train, y_test = y_train.iloc[train_idx], y_train.iloc[test_idx]

In [ ]:
space = {'n_estimators': hp.choice("n_estimators", list(range(100, 1600, 100))),
        "booster": hp.choice('booster', ['gbtree', 'gblinear', 'dart']),
        "eval_metric": hp.choice('eval_metric', ['auc', 'logloss', 'error']),
        "objective": hp.choice('objective', ['binary:logistic', 'reg:logistic', 'binary:hinge']),
        "sampling_method": hp.choice('sampling_method', ['uniform', 'gradient_based']),
        "tree_method": hp.choice('tree_method', ['auto', 'approx']),
        "max_depth": hp.choice('max_depth', list(range(1, 11))),
        "reg_lambda": hp.choice('reg_lambda', np.logspace(-2, 1, 20)),
        "gamma": hp.choice('gamma', np.logspace(-2, 1, 20)),
        "min_child_weight": hp.choice('min_child_weight', np.logspace(-2, 1, 20)),
        "learning_rate": hp.choice('learning_rate', np.logspace(-3, 1, 20)),
        'seed': 0
    }

In [ ]:
def objective(space):

	clf = XGBClassifier(**space)

	evaluation = [(X_train, y_train), 
				(X_test, y_test)]

	clf.fit(X_train,
		y_train,
		eval_set=evaluation,
		early_stopping_rounds=10,
		verbose=False)

	pred = clf.predict(X_test)
	accuracy = accuracy_score(y_test, pred>0.5)

	scores.append(accuracy)
		
	return {'loss': -1 * accuracy,
			'status': STATUS_OK}

In [ ]:
start = time()

trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 150,
                        trials = trials)

end = time()

In [ ]:
print(f"Optimization time in seconds: {round(end-start, 2)}")
print()

print(f"Best score: {round(max(scores), 2)}")
print()

print("The best hyperparameters are")
best_params = space_eval(space=space, hp_assignment=best_hyperparams)
pprint(best_params)

print()
clf = XGBClassifier(**best_params)
clf.fit(X_train, y_train)
pred = clf.predict(X_test_oos)
print(f"Predicted labels:\n{np.unique(pred, return_counts=True)}")